In [ ]:
#Here we will do the coding part of analysis of video data in the project (student performance analysis).

In [ ]:
!pip install opencv-python
!pip install deepface

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2

In [ ]:
import os
video_data1= "/content/drive/MyDrive/SPA videos/MM_Video/Vineet.mp4"

In [ ]:
import numpy as np

In [ ]:
def preprocess_video(video_data1, frame_size=(224, 224), normalize=True, to_grayscale=False):
    cap = cv2.VideoCapture(video_data1)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_data1}")
        return []
    processed_frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, frame_size)

        if to_grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if normalize:
            frame = frame.astype('float32') / 255.0

        processed_frames.append(frame)

    cap.release()
    return processed_frames


frames = preprocess_video(video_data1, frame_size=(224, 224), normalize=True, to_grayscale=False)
frames = np.array(frames)
print(f"Processed {frames.shape[0]} frames with shape {frames.shape[1:]}")

Processed 2548 frames with shape (224, 224, 3)


In [ ]:
import cv2

video_path = '/content/drive/MyDrive/SPA videos/MM_Video/Vineet.mp4'
vid1 = cv2.VideoCapture(video_path)

fps = int(vid1.get(cv2.CAP_PROP_FPS))
print("Frames per second:", fps)

i = 0
n = 0
interval_seconds = 15
target_seconds = [5, 10, 15]
frames_to_save = 3
time_limit_seconds = 75

for target_second in target_seconds:
    target_frame = target_second * fps
    while True:
        success, frame = vid1.read()
        if not success:
            break

        # Stop if we have passed the 75 seconds limit
        if n / fps > time_limit_seconds:
            break

        if (n % (interval_seconds * fps)) == target_frame:
            for _ in range(frames_to_save):
                frame_filename = "/content/drive/MyDrive/SPA videos/MM_pics/vineet pics/Vineet{}.jpg".format(i)
                cv2.imwrite(frame_filename, frame)
                i += 1
                success, frame = vid1.read()
                n += 1
                if not success or n / fps > time_limit_seconds:
                    break

        n += 1

vid1.release()
print("Extracted and saved {} frames.".format(i))



Frames per second: 30
Extracted and saved 15 frames.


In [ ]:
!pip install deepface

from deepface import DeepFace
import cv2

dominant_emotion_list = []


for i in range (18):
    image_path = "/content/drive/MyDrive/SPA videos/MM_pics/vineet pics/Vineet{}.jpg".format(i)

    image = cv2.imread(image_path)
    if image is not None:
        try:

            analysis = DeepFace.analyze(img_path=image_path, actions=['emotion'], enforce_detection=False)

            if isinstance(analysis, list) and len(analysis) > 0:
                emotion_data = analysis[0]['emotion']
            else:
                emotion_data = analysis['emotion']


            dominant_emotion = max(emotion_data, key=emotion_data.get)
            dominant_emotion_list.append(dominant_emotion)
        except Exception as e:

            dominant_emotion_list.append(None)
            print(f"Error processing frame {i+1}: {e}")
    else:
        dominant_emotion_list.append(None)


for i, dominant_emotion in enumerate(dominant_emotion_list):
    print(f"Frame {i+1}: {dominant_emotion}")


Frame 1: neutral
Frame 2: neutral
Frame 3: neutral
Frame 4: angry
Frame 5: neutral
Frame 6: neutral
Frame 7: neutral
Frame 8: neutral
Frame 9: neutral
Frame 10: angry
Frame 11: neutral
Frame 12: angry
Frame 13: angry
Frame 14: angry
Frame 15: neutral
Frame 16: None
Frame 17: None
Frame 18: None


In [ ]:
from collections import Counter

# Count occurrences of each emotion
emotion_counter = Counter(dominant_emotion_list)

# Print the summary of dominant emotions
print("Summary of Dominant Emotions:")
for emotion, count in emotion_counter.items():
    print(f"{emotion}: {count} frames")

# Optional: Remove None from the summary if not needed
if None in emotion_counter:
    del emotion_counter[None]

# If you want to analyze the correlation, provide the performance data, and we can proceed from there.


Summary of Dominant Emotions:
neutral: 12 frames
happy: 3 frames
None: 3 frames


In [ ]:
import pandas as pd

# Example data (you should replace this with your actual data)
emotions = [
    'neutral', 'neutral', 'neutral',
    'angry', 'neutral', 'neutral',
    'neutral', 'neutral', 'neutral',
    'angry', 'neutral', 'angry',
    'angry', 'angry', 'neutral'
]

# Create a DataFrame
df = pd.DataFrame({
    'frame': range(1, 16),
    'emotion': emotions,
    'question': [1]*3 + [2]*3 + [3]*3 + [4]*3 + [5]*3
})

# Aggregate emotions by question
emotion_summary = df.groupby('question')['emotion'].value_counts().unstack().fillna(0)

print(emotion_summary)


emotion   angry  neutral
question                
1           0.0      3.0
2           1.0      2.0
3           0.0      3.0
4           2.0      1.0
5           2.0      1.0


In [ ]:
import pandas as pd
import numpy as np
# Example emotion data
emotion_data = {
    'frame': range(1, 16),
    'emotion': [
          'neutral', 'neutral', 'neutral',
    'angry', 'neutral', 'neutral',
    'neutral', 'neutral', 'neutral',
    'angry', 'neutral', 'angry',
    'angry', 'angry', 'neutral'
    ],
    'question': [1]*3 + [2]*3 + [3]*3 + [4]*3 + [5]*3
}

emotion_df = pd.DataFrame(emotion_data)

# Updated participant's answers data
answers_data = {
    'Question': [1, 2, 3, 4, 5],
    'Answer': ['b', 'c', 'd', 'c', 'a'],
    'Correct': [1, 1,1,0, 1]
}

answers_df = pd.DataFrame(answers_data)

# Merge the emotion data with the participant's answers
merged_df = emotion_df.merge(answers_df, how='left', left_on='question', right_on='Question')

# Label the data
merged_df['label'] = merged_df['Correct'].apply(lambda x: 'correct' if x == 1 else 'incorrect')

print(merged_df)



    frame  emotion  question  Question Answer  Correct      label
0       1  neutral         1         1      b        1    correct
1       2  neutral         1         1      b        1    correct
2       3  neutral         1         1      b        1    correct
3       4    angry         2         2      c        1    correct
4       5  neutral         2         2      c        1    correct
5       6  neutral         2         2      c        1    correct
6       7  neutral         3         3      d        1    correct
7       8  neutral         3         3      d        1    correct
8       9  neutral         3         3      d        1    correct
9      10    angry         4         4      c        0  incorrect
10     11  neutral         4         4      c        0  incorrect
11     12    angry         4         4      c        0  incorrect
12     13    angry         5         5      a        1    correct
13     14    angry         5         5      a        1    correct
14     15 

In [ ]:
# combine every 3 frames as P1 to p5. i just need the emotion and question and answer and correct and label columns

combined_df = merged_df.groupby(np.arange(len(merged_df)) // 3).agg(
    emotion=('emotion', lambda x: list(x)),
    question=('question', 'first'),
    answer=('Answer', 'first'),
    correct=('Correct', 'first'),
    label=('label', 'first')
)


combined_df.index.name = 'part'


print(combined_df)


                          emotion  question answer  correct      label
part                                                                  
0     [neutral, neutral, neutral]         1      b        1    correct
1       [angry, neutral, neutral]         2      c        1    correct
2     [neutral, neutral, neutral]         3      d        1    correct
3         [angry, neutral, angry]         4      c        0  incorrect
4         [angry, angry, neutral]         5      a        1    correct


In [ ]:
# in the emotion column just keep the dominent emotion


def find_dominant_emotion(emotion_list):
  emotion_counts = Counter(emotion_list)
  return emotion_counts.most_common(1)[0][0]


combined_df['emotion'] = combined_df['emotion'].apply(find_dominant_emotion)


print(combined_df)


      emotion  question answer  correct      label
part                                              
0     neutral         1      b        1    correct
1     neutral         2      c        1    correct
2     neutral         3      d        1    correct
3       angry         4      c        0  incorrect
4       angry         5      a        1    correct


In [ ]:

# update the abouve data into this file "/content/drive/MyDrive/video_data.xlsx"

# Export the combined DataFrame to an Excel file
combined_df.to_excel('/content/drive/MyDrive/video_data.xlsx', index=False)
